In [6]:
input_video = "images/bbaf2n.mpg"

import cv2

v = cv2.VideoCapture(input_video)
i = 0
while v.isOpened():
  _, frame = v.read()
  if frame is None: break
  cv2.imwrite(f"videos/grid_example/grid-{i}.jpg", frame)
  i+=1

[mpeg1video @ 0x6cb41c0] ac-tex damaged at 19 10
[mpeg1video @ 0x6cb41c0] Warning MVs not available


In [1]:
from matplotlib import pyplot as plt
import cv2

def show_figures(imgs, name = None, tiny_subimage = True):
  qtd = len(imgs)
  if qtd == 2 and tiny_subimage:
    fig, axs = plt.subplots(1,2,figsize=(15,10), gridspec_kw={'width_ratios': [2, 1]})
    for i in range(qtd):
      axs[i].axis("off")
      axs[i].imshow(cv2.cvtColor(imgs[i], cv2.COLOR_BGR2RGB))
    fig.savefig(name, bbox_inches='tight')
    plt.close()
  else:
    assert qtd > 0, "ta de sacanagem ne"
    plt.figure(figsize=(10*qtd, 10))
    for i in range(len(imgs)):
      plt.subplot(1, qtd, i+1)
      plt.axis("off")
      plt.imshow(cv2.cvtColor(imgs[i], cv2.COLOR_BGR2RGB))
    plt.savefig(name, bbox_inches='tight')
    plt.close()

import numpy as np

def draw_triangle(frame, points):
  x = lambda a: (int(a[0]), int(a[1]))
  triangle1 = frame.copy()
  mid_eyes = np.mean([points[0][0], points[1][0]]), np.mean([points[0][1], points[1][1]])
  cv2.circle(triangle1, x(points[0]), 5, (255, 0, 0), 10)
  cv2.circle(triangle1, x(points[1]), 5, (255, 0, 0), 10)
  cv2.circle(triangle1, x(points[2]), 5, (255, 0, 0), 10)
  triangle2 = triangle1.copy()
  cv2.circle(triangle2, x(mid_eyes), 5, (255, 0, 0), 10)
  cv2.line(triangle2, x(points[0]), x(points[1]), (255, 0, 0), 2)
  triangle3 = triangle2.copy()
  cv2.line(triangle3, x(mid_eyes), x(points[2]), (255, 0, 0), 2)

  return triangle1, triangle2, triangle3

In [2]:
import cv2
from preprocessing.mouth_extraction import FaceFrame

input_video = "images/bbac5n.mpg"

j = 0
v = cv2.VideoCapture(input_video)
while v.isOpened():
  _, frame = v.read()
  if frame is None: break

  f_obj = FaceFrame(frame)

  circles = frame.copy()

  for i in range(len(f_obj.lm)):
    if i in list(range(36, 68)):
      cv2.circle(circles, f_obj.lm[i], 5, (255, 0, 0), 1)
      
    else:
      cv2.circle(circles, f_obj.lm[i], 5, (100, 255, 0), 1)
    
  l_eye, r_eye, mouth = [f_obj.l_eye, f_obj.r_eye, f_obj.mouth]

  _, _, centers = draw_triangle(frame, [l_eye, r_eye, mouth])

  f_obj.transform()

  _, _, affine = draw_triangle(f_obj.img, [f_obj.l_eye, f_obj.r_eye, f_obj.mouth])

  mouth_crop = f_obj.get_mouth_img(70, "crop")
  temp_resize = f_obj.get_mouth_img(70, "resize")
  mouth_resize = cv2.resize(temp_resize, (100, 50))

  m = [int(i) for i in f_obj.mouth]
  tl = m[0]-70, m[1]-70//2
  br = m[0]+70, m[1]+70//2
  temp = f_obj.img.copy()
  square_crop = cv2.rectangle(temp, tl, br, (255, 0, 0))

  tl, br = f_obj.get_mouth_corners()
  temp = f_obj.img.copy()
  square_resize = cv2.rectangle(temp, tl, br, (255, 0, 0))

  show_figures([frame], f"videos/preproc_1_base/base-{j}.jpg")
  show_figures([circles], f"videos/preproc_2_roi/roi-{j}.jpg")
  show_figures([centers], f"videos/preproc_3_centers/centers-{j}.jpg")
  show_figures([centers, affine], f"videos/preproc_4_affine/affine-{j}.jpg", False)
  show_figures([square_crop, mouth_crop], f"videos/preproc_5_crop/crop-{j}.jpg")
  show_figures([square_resize, mouth_resize], f"videos/preproc_6_resize/resize-{j}.jpg")

  j+=1

In [10]:
v = cv2.VideoCapture(input_video)
_, frame = v.read()
f_obj = FaceFrame(frame)

t1, t2, t3 = draw_triangle(f_obj.img, [f_obj.l_eye, f_obj.r_eye, f_obj.mouth])

show_figures([t1], f"images/constructing_triangle_1.jpg")
show_figures([t2], f"images/constructing_triangle_2.jpg")
show_figures([t3], f"images/constructing_triangle_3.jpg")